# 用纯属性取代get和set方法

In [1]:
import logging

有的开发者可能会在类中明确地实现getter和setter方法。

In [2]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

In [3]:
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After:  %5r' % r0.get_ohms())

Before: 50000.0
After:  10000.0


对于自增的代码，这种用法尤其显得麻烦。

In [4]:
r0.set_ohms(r0.get_ohms() + 5e3)

改进方法：python语言，基本上不需要手工实现setter或getter方法，而是应该先从简单的public属性开始写起。

In [5]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

In [6]:
r1 = Resistor(50e3)
r1.ohms = 10e3
print('%r ohms, %r volts, %r amps' %
      (r1.ohms, r1.voltage, r1.current))

10000.0 ohms, 0 volts, 0 amps


对于自增操作，可以很方便的使用。

In [7]:
r1.ohms += 5e3

**示例：**下面这个子类继承自Resistor，在给voltage属性赋值的时候，还会同时修改current属性。

**注意：**setter和getter方法的名称必须与相关属性相符。

In [8]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

设置voltage属性时，将会执行名为voltage的setter方法，该方法会更新本对象的current属性，令其与电压和电阻相匹配。

In [9]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After:  %5r amps' % r2.current)

Before:     0 amps
After:   0.01 amps


为属性指定setter方法时，也可以在方法里面做类型验证及数值验证。  
**示例：**保证传入的电阻值总是大于0欧姆。

In [10]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

In [11]:
try:
    r3 = BoundedResistance(1e3)
    r3.ohms = 0
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-11-500ef67ee041>", line 3, in <module>
    r3.ohms = 0
  File "<ipython-input-10-b77a86c858c8>", line 12, in ohms
    raise ValueError('%f ohms must be > 0' % ohms)
ValueError: 0.000000 ohms must be > 0


In [12]:
try:
    BoundedResistance(-5)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-12-693d2cd5f3c8>", line 2, in <module>
    BoundedResistance(-5)
  File "<ipython-input-10-b77a86c858c8>", line 3, in __init__
    super().__init__(ohms)
  File "<ipython-input-5-f2740f70eecf>", line 3, in __init__
    self.ohms = ohms
  File "<ipython-input-10-b77a86c858c8>", line 12, in ohms
    raise ValueError('%f ohms must be > 0' % ohms)
ValueError: -5.000000 ohms must be > 0


**示例：**可以用@property来防止父类的属性遭到修改。

In [13]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [14]:
try:
    r4 = FixedResistance(1e3)
    r4.ohms = 2e3
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-14-23362e41a337>", line 3, in <module>
    r4.ohms = 2e3
  File "<ipython-input-13-e74e36697024>", line 12, in ohms
    raise AttributeError("Can't set attribute")
AttributeError: Can't set attribute


@property的最大缺点在于：和属性相关的方法，只能在子类里面共享，而与之无关的其他类，则无法复用同一份实现代码。

**注意：**不应该在某属性的getter方法里面修改其他属性的值。

In [15]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

In [16]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After:  %5r' % r7.voltage)

Before:     0
After:    0.1


出现以上的原因是构建的时候，ohms=10，然后current=0.01，voltage=0，后来由于执行了r7.ohms导致voltage=10*0.01=0.1，故打印出的voltage=0.1